## 음식메뉴 코사인 유사도 

**목표**<br>
챗봇에서 음식메뉴를 입력 받을 때, 데이터셋에 있는 음식메뉴와 정확하게 일치하지 않아도 코사인 유사도 모델을 통해 비슷한 음식메뉴를 선택할 수 있다.

- 가게리스트 데이터의 Category1, Category2, menu 컬럼을 이용해 food 데이터셋을 만든다.
- 위 데이터셋을 이용해 각 음식메뉴별 코사인 유사도를 나타내는 모델을 만든다.


#### 코사인 유사도 설명 참고자료
- https://wikidocs.net/24603

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Mecab

### 1. 음식메뉴 데이터 불러오기 

In [3]:
df_foods = pd.read_csv('./questions/foods_bio.csv', encoding='utf-8', index_col=0)
df_foods.head(3)

,food,mecab,mecab_tag,okt,okt_tag
0,BBQ,['BBQ'],['B-FOD'],['BBQ'],['B-FOD']
1,BBQ샐러드,"['BBQ', '샐러드']","['B-FOD', 'I-FOD']","['BBQ', '샐러드']","['B-FOD', 'I-FOD']"
2,가라아게,['가라아게'],['B-FOD'],['가라아게'],['B-FOD']


In [2]:
# 가게리스트 데이터 불러오기
df_restaurants = pd.read_csv('./questions/가게리스트_카테고리_최종.csv',encoding='utf-8',index_col=0)
df_restaurants.head(2)

,storeid,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
0,13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,숯불구이,육회비빔밥,24,2
1,13995,착한소장수,착한소장수,광주,동구,광주,한식,갈비,갈비살/안창살/살치살/왕갈비탕,4155,7


In [6]:
# Category1, Category2, menu 컬럼 저장
df_foods = df_restaurants.groupby(['Category1','Category2','menu'], as_index=False).count()\
           [['Category1','Category2','menu']]
df_foods.shape

(916, 3)

In [7]:
df_foods.head(10)

,Category1,Category2,menu
0,기타,디저트,과일주스/ 수박주스
1,기타,디저트,꿀빵
2,기타,디저트,미용실
3,기타,디저트,바게트
4,기타,디저트,반찬가게
5,기타,디저트,보리빵
6,기타,디저트,빵
7,기타,디저트,아이스크림
8,기타,디저트,알수없음
9,기타,디저트,유원지


In [10]:
# Category1 + Category2 + menu를 합쳐서 저장
df_foods['food'] = df_foods['Category1'] + ' ' + df_foods['Category2'] + ' ' + df_foods['menu']
df_foods.head(10)

,Category1,Category2,menu,food
0,기타,디저트,과일주스/ 수박주스,기타 디저트 과일주스/ 수박주스
1,기타,디저트,꿀빵,기타 디저트 꿀빵
2,기타,디저트,미용실,기타 디저트 미용실
3,기타,디저트,바게트,기타 디저트 바게트
4,기타,디저트,반찬가게,기타 디저트 반찬가게
5,기타,디저트,보리빵,기타 디저트 보리빵
6,기타,디저트,빵,기타 디저트 빵
7,기타,디저트,아이스크림,기타 디저트 아이스크림
8,기타,디저트,알수없음,기타 디저트 알수없음
9,기타,디저트,유원지,기타 디저트 유원지


### 2. `TfidfVectorizer`를 이용하여 벡터화

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
KOREAN_RE = re.compile('[^ ㄱ-ㅣ가-힣]+')

# 불용어는 챗봇 모델과 동일
STOPWORDS = set(['은','는','이','가','하','아','것','들','의','그','수','한','나','같','그렇'
                ,'문제','그리고','크','중','나오','지금','생각하','집','어떤','명','생각','이런'
                ,'인','지','을','를','에','스러운','스러워','주','할','만','게','도','져','된','로','고','던','로운','면서'
                ,'사실','이렇','점','싶','말','좀','식당','가게','집','음식점'
                ,'는지','나요','해요','해','는가요','삼','게요','예','는가','습니까','죠','려고요','는지요','서요','였어요','겠'
                ,'인가요','요' '라는','데','해서','세요','어요','을까요','건가요','겠죠','실래요','네요','으세요','지요','인데요'
                ,'드려요','려구요','합니다'])

# 각 음식메뉴를 mecab 형태소 분석하기 위한 함수
def text_prepare(text):
    
    # 한글/공백이 아닌 기타 문자는 space로 대체 (ex. 짜장면/짬뽕 -> 짜장면 짬뽕)
    text = KOREAN_RE.sub(' ',text)
    
    # Mecab 토크나이저
    mecab = Mecab()
    
    # mecab으로 text를 형태소 단위로 나누고 불용어를 제거
    text = ' '.join(token for token in mecab.morphs(text) if token not in STOPWORDS)

    return text

In [13]:
# 지역명에 mecab 형태소 분석 적용
df_foods['food_prep'] = df_foods.food.apply(text_prepare)
df_foods.head(10)

,Category1,Category2,menu,food,food_prep
0,기타,디저트,과일주스/ 수박주스,기타 디저트 과일주스/ 수박주스,기타 디저트 과일 주스 수박 주스
1,기타,디저트,꿀빵,기타 디저트 꿀빵,기타 디저트 꿀 빵
2,기타,디저트,미용실,기타 디저트 미용실,기타 디저트 미용실
3,기타,디저트,바게트,기타 디저트 바게트,기타 디저트 바게트
4,기타,디저트,반찬가게,기타 디저트 반찬가게,기타 디저트 반찬
5,기타,디저트,보리빵,기타 디저트 보리빵,기타 디저트 보리 빵
6,기타,디저트,빵,기타 디저트 빵,기타 디저트 빵
7,기타,디저트,아이스크림,기타 디저트 아이스크림,기타 디저트 아이스크림
8,기타,디저트,알수없음,기타 디저트 알수없음,기타 디저트 알 없 음
9,기타,디저트,유원지,기타 디저트 유원지,기타 디저트 유원지


In [14]:
# TF-IDF vectorizer 생성
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2) # unigram, bigram 
                                  ,token_pattern='(\S+)') # 공백을 제거한 모든 문자/숫자/특수문자

# 벡터화할 데이터를 저장
y = df_foods.food_prep.values

# 벡터화된 데이터의 지역명을 저장
y_food = df_foods.food.values

# vectorizer를 y 데이터로 fitting
tfidf_vectorizer.fit(y)

# fitting 된 vectorizer 확인
tfidf_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(\\S+)', tokenizer=None,
                use_idf=True, vocabulary=None)

In [16]:
# tfidf에 사용된 형태소 단어 확인 : 916개 음식메뉴를 나타내기 위해 총 2,203개 형태소가 사용되었음
tfidf_vocabs = tfidf_vectorizer.vocabulary_
len(tfidf_vocabs)

2203

In [17]:
# 형태소 단어 미리보기
tfidf_vocabs

{'기타': 238,
 '디저트': 507,
 '과일': 137,
 '주스': 1610,
 '수박': 1168,
 '기타 디저트': 239,
 '디저트 과일': 508,
 '과일 주스': 138,
 '주스 수박': 1611,
 '수박 주스': 1169,
 '꿀': 290,
 '빵': 928,
 '디저트 꿀': 509,
 '꿀 빵': 291,
 '미용실': 699,
 '디저트 미용실': 510,
 '바게트': 714,
 '디저트 바게트': 511,
 '반찬': 730,
 '디저트 반찬': 512,
 '보리': 775,
 '디저트 보리': 513,
 '보리 빵': 776,
 '디저트 빵': 514,
 '아이스크림': 1302,
 '디저트 아이스크림': 515,
 '알': 1310,
 '없': 1355,
 '음': 1478,
 '디저트 알': 516,
 '알 없': 1311,
 '없 음': 1356,
 '유원지': 1463,
 '디저트 유원지': 517,
 '중국': 1618,
 '디저트 중국': 518,
 '중국 빵': 1619,
 '캠': 1760,
 '핌': 1966,
 '장': 1525,
 '디저트 캠': 519,
 '캠 핌': 1761,
 '핌 장': 1967,
 '테마파크': 1866,
 '디저트 테마파크': 520,
 '펜션': 1937,
 '디저트 펜션': 521,
 '풀': 1947,
 '빌': 924,
 '라': 548,
 '디저트 풀': 522,
 '풀 빌': 1948,
 '빌 라': 925,
 '뷔페': 874,
 '고기': 63,
 '기타 뷔페': 240,
 '뷔페 고기': 875,
 '국수': 196,
 '뷔페 국수': 876,
 '떡볶이': 531,
 '뷔페 떡볶이': 877,
 '바베큐': 718,
 '뷔페 바베큐': 878,
 '뷔페 뷔페': 879,
 '샐러드': 1011,
 '뷔페 샐러드': 880,
 '소고기': 1125,
 '뷔페 소고기': 881,
 '숙박': 1184,
 '시설': 1259,
 '뷔페 숙박': 882,
 '숙

In [18]:
# y 데이터 벡터화
y_vectorized = tfidf_vectorizer.transform(y)

# 벡터화된 희소행렬 확인 
# (916*2203 행렬, 916개 음식메뉴가 총 2203개의 형태소 벡터를 이용하여 벡터화되었음)
# (6335개 element가 있으므로, 한 행당 평균 7개 정도의 element가 있음)
y_vectorized

<916x2203 sparse matrix of type '<class 'numpy.float64'>'
	with 6335 stored elements in Compressed Sparse Row format>

### 3. cosine 유사도 모델 생성 : 데이터 내부
- "y데이터 내부"에서 서로 비슷한 음식메뉴는 무엇인지 찾는 모델 생성
- [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html) 이용

In [19]:
from sklearn.metrics.pairwise import linear_kernel

# y_vectorized 데이터를 이용해 linear_kernel 행렬 생성
cosine_sim = linear_kernel(y_vectorized, y_vectorized)

In [20]:
cosine_sim.shape

(916, 916)

In [28]:
def get_similar_indices(idx, cosine_sim=cosine_sim):
    """
        idx를 "y데이터 내부"에서 서로 비슷한 음식메뉴는 무엇인지 찾는 모델 생성
    """

    # 모든 음식메뉴에 대해서 해당 idx와의 유사도를 구한다. ((index, linear_kernel값)의 list 형태)
    sim_scores = list(enumerate(cosine_sim[idx])) 

    # 유사도에 따라 음식메뉴를 정렬한다. (key 파라미터는 정렬기준을 선택)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 0 이상인 것만 sim_list에 저장한다.
    sim_list=[]
    for i in range(len(sim_scores)):
        if sim_scores[i][1] > 0:
            sim_list.append(sim_scores[i])
            
    # sim_list에서 유사도가 가장 높은 3개만 저장한다.
    sim_list = sim_list[:3]
            
    # sim_list의 인덱스를 저장한다.
    sim_indices = [i[0] for i in sim_list]

    # 5개의 음식메뉴를 list 타입으로 반환한다.
    return sim_indices

In [30]:
test_idxs = [10,40,60,80,100,130,150,180,200]
for idx in test_idxs:
    test_food = y_food[idx]
    sim_indices = get_similar_indices(idx)
    sim_food = [y_food[idx] for idx in sim_indices]
    print("'",test_food,"'",'가장 유사한 지역명 5개:', sim_food)

' 기타 디저트 중국빵 ' 가장 유사한 지역명 5개: ['기타 디저트 중국빵', '기타 디저트 빵', '기타 디저트 꿀빵']
' 기타 호프 맥주/ 한치 ' 가장 유사한 지역명 5개: ['기타 호프 맥주/ 한치', '기타 호프 맥주', '기타 호프 맥주/ 치킨']
' 기타아시아식 인도 머턴커리 ' 가장 유사한 지역명 5개: ['기타아시아식 인도 머턴커리', '기타아시아식 인도 인도요리/ 치킨커리', '기타아시아식 인도 쌀국수']
' 서양식 멕시코 뷔페 ' 가장 유사한 지역명 5개: ['서양식 멕시코 뷔페', '서양식 멕시코 서양식', '서양식 멕시코 피자']
' 서양식 브런치 브런치바비큐플래터/ 스테이크샐러드 ' 가장 유사한 지역명 5개: ['서양식 브런치 브런치바비큐플래터/ 스테이크샐러드', '서양식 브런치 브런치', '서양식 브런치 샐러드']
' 서양식 브런치 퀘사디아 ' 가장 유사한 지역명 5개: ['서양식 브런치 퀘사디아', '서양식 멕시코 멕시코요리/ 타코/ 퀘사디아', '서양식 브런치 브런치']
' 서양식 스테이크 칵테일/ 와인/ 샐러드 ' 가장 유사한 지역명 5개: ['서양식 스테이크 칵테일/ 와인/ 샐러드', '서양식 스테이크 와인/ 칵테일', '서양식 스테이크 와인']
' 서양식 피자/ 파스타 고르곤졸라 ' 가장 유사한 지역명 5개: ['서양식 피자/ 파스타 고르곤졸라', '서양식 피자/ 파스타 피자/ 파스타', '서양식 피자/ 파스타 파스타']
' 서양식 피자/ 파스타 빠네 ' 가장 유사한 지역명 5개: ['서양식 피자/ 파스타 빠네', '서양식 피자/ 파스타 피자/ 파스타', '서양식 피자/ 파스타 파스타']


`locations_bio.csv` 파일 내부의 지역명에 대해서는 정확하게 유사한 지역명을 예측했다.

### 4. cosine 유사도 모델 생성 : 데이터 외부
- df_foods 데이터에 없던 음식메뉴가 입력되었을 때, df_foods 내에서 가장 비슷한 음식메뉴는 무엇인지 찾는 모델 생성
- [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 이용

#### 데이터에 없던 새로운 음식명을 테스트

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
def get_similar_food(new_food, tfidf_vectorizer, y_vectorized):
    """
        데이터에 없는 새로운 지역명 new_food을 입력 받고, 
        new_food과 가장 유사한 지역명 3개와 각각의 유사도를 데이터프레임 형태로 반환
    """
    
    # new_food 형태소 분석
    new_food_mecab = [text_prepare(new_food)]
    
    # new_food 벡터화
    new_food_vectorized = tfidf_vectorizer.transform(new_food_mecab)
    
    # 유사한 메뉴와 유사도를 저장
    sim_food = {}

    for i in range(y_vectorized.shape[0]):
        curr_sim = cosine_similarity(y_vectorized[i], new_food_vectorized)[0,0]
        if curr_sim > 0.2:
            sim_food[y_food[i]] = curr_sim

    sim_food = pd.DataFrame(data=sim_food.values()
                           ,index=sim_food.keys()
                           ,columns=['cs']).sort_values('cs',ascending=False)

    return sim_food[:3]

In [59]:
y_food

array(['기타 디저트 과일주스/ 수박주스', '기타 디저트 꿀빵', '기타 디저트 미용실', '기타 디저트 바게트',
       '기타 디저트 반찬가게', '기타 디저트 보리빵', '기타 디저트 빵', '기타 디저트 아이스크림',
       '기타 디저트 알수없음', '기타 디저트 유원지', '기타 디저트 중국빵', '기타 디저트 캠핌장',
       '기타 디저트 테마파크', '기타 디저트 펜션', '기타 디저트 풀빌라', '기타 뷔페 고기', '기타 뷔페 국수',
       '기타 뷔페 떡볶이', '기타 뷔페 바베큐', '기타 뷔페 뷔페', '기타 뷔페 샐러드', '기타 뷔페 소고기',
       '기타 뷔페 숙박시설', '기타 뷔페 스테이크', '기타 뷔페 웨딩홀', '기타 뷔페 한식', '기타 뷔페 한식뷔페',
       '기타 카페 술', '기타 카페 차', '기타 카페 카페', '기타 카페 커피', '기타 카페 케이크',
       '기타 포장마차 주류', '기타 포장마차 포장마차', '기타 호프 골뱅이무침', '기타 호프 막걸리',
       '기타 호프 막걸리/ 주류', '기타 호프 맥주', '기타 호프 맥주/ 주류', '기타 호프 맥주/ 치킨',
       '기타 호프 맥주/ 한치', '기타 호프 맥주/주류', '기타 호프 소주', '기타 호프 술', '기타 호프 알수없음',
       '기타 호프 정종/대포집/소주방', '기타 호프 주류', '기타 호프 주류/ 오뎅탕', '기타 호프 주점',
       '기타 호프 치킨', '기타 호프 파전', '기타 호프 호프/통닭', '기타아시아식 베트남 샤브샤브',
       '기타아시아식 베트남 쌀국수', '기타아시아식 베트남 쌀국수/ 반쎄오', '기타아시아식 베트남 쌀국수/ 분짜',
       '기타아시아식 베트남 쌀국수/ 짜조', '기타아시아식 베트남 월남쌈/ 샤브샤브',
       '기타아시아식 베트남 팟타이/ 쌀국수', '기타아시아식 인도 나시고랭/ 미고랭', '기

In [62]:
type(y_food), y_food.shape

(numpy.ndarray, (916,))

In [60]:
y_vectorized

<916x2203 sparse matrix of type '<class 'numpy.float64'>'
	with 6335 stored elements in Compressed Sparse Row format>

In [57]:
test_foods = [
    '한식','중식','중국식','서양식','일식','일본식','태국요리','태국음식','베트남','쌀국수'
    ,'설렁탕','곰탕','갈비탕','갈비찜','멕시칸','멕시코','닭볶음','오징어','순대','순대국밥'
    ,'퓨전','철판구이','꿀떡','호빵','생선구이','국밥','고등어조림','갈치조림','고등어구이'
    ,'타코','파스타','피자','짜장면','탕수육','마라탕','오메기떡','회','방어회','스시','사시미'
    ,'복어','흑돼지','소갈비'
]

print('가장 유사한 메뉴 3가지')
for food in test_foods:
    print(food,'\t:',get_similar_food(food,tfidf_vectorizer,y_vectorized).index.tolist())
    print('\tscore:',get_similar_food(food,tfidf_vectorizer,y_vectorized)['cs'].tolist())

가장 유사한 메뉴 3가지
한식 	: ['한식 게 게', '한식 해장국 한식']
	score: [1.0, 0.2666188853273465]
중식 	: []
	score: []
중국식 	: ['중국식 탕수육 유린기', '중국식 탕수육 훠궈', '중국식 짜장면/ 짬뽕 피자']
	score: [0.3505920216161386, 0.3454059532897735, 0.33259184881901305]
서양식 	: ['서양식 멕시코 서양식', '서양식 브런치 커피', '서양식 브런치 아메리카노']
	score: [0.4772297134005853, 0.28010968089652644, 0.26304788783384425]
일식 	: ['일식 일식 회 생선회', '일식 일식 회 활어', '일식 일식 회 초밥']
	score: [0.5268033545106742, 0.5268033545106742, 0.5173082891882225]
일본식 	: []
	score: []
태국요리 	: ['기타아시아식 태국 태국요리', '기타아시아식 태국 태국요리전문', '기타아시아식 태국 태국요리/ 갈비국수/ 팟타이']
	score: [0.6798904932847614, 0.6087103550130296, 0.5317343500512843]
태국음식 	: ['기타아시아식 태국 태국요리', '기타아시아식 태국 태국요리전문', '기타아시아식 태국 태국요리/ 갈비국수/ 팟타이']
	score: [0.3777007210223718, 0.33815789785710526, 0.29539528702104556]
베트남 	: ['기타아시아식 베트남 쌀국수', '기타아시아식 베트남 샤브샤브', '기타아시아식 베트남 팟타이/ 쌀국수']
	score: [0.3718980755293194, 0.3648241407457343, 0.31011366241138383]
쌀국수 	: ['기타아시아식 베트남 쌀국수', '기타아시아식 인도 쌀국수', '기타아시아식 베트남 팟타이/ 쌀국수']
	score: [0.38054

**pickle 저장**

In [58]:
import pickle

with open('./cosine_food_tfidf_vectorizer.pickle', "wb") as f:
    pickle.dump(tfidf_vectorizer, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_food_y_food.pickle', "wb") as f:
    pickle.dump(y_food, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_food_y_vectorized.pickle', "wb") as f:
    pickle.dump(y_vectorized, f, pickle.HIGHEST_PROTOCOL)
with open('./csine_food_cosine_sim.pickle', "wb") as f:
    pickle.dump(cosine_sim, f, pickle.HIGHEST_PROTOCOL)

**클래스 정리**